In [64]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np
import re

wasteCollectionDate = pd.concat([pd.read_csv('../'+i) for i in os.listdir('..') if i.startswith('GBD')])\
.sort_values(['binInfo_bin_type', 'postcode'])\
.drop_duplicates()\
.reset_index(drop = True)

wasteCollectionDate['postcode'] = wasteCollectionDate.postcode.astype(str).str.zfill(4)

wasteCollectionDate['i'] = wasteCollectionDate.groupby(['postcode', 'binInfo_bin_type']).transform('sum')

correctStrings = wasteCollectionDate.loc[~wasteCollectionDate.i.str.contains('Every (.+)dayEvery')]

wrongStrings = wasteCollectionDate.loc[wasteCollectionDate.i.str.contains('Every (.+)dayEvery')]\
.drop_duplicates(['binInfo_bin_type', 'postcode', 'i'])

wrongStrings['i'] = wrongStrings.i.apply(lambda x: re.sub('day(Every )', 'day or ', x))\
.apply(lambda x: re.sub('day or Second ', 'day or ', x))\
.apply(lambda x: if re.findall('[Friday]'))

wasteCollectionDate = pd.concat([correctStrings, wrongStrings]).drop('collection_time', axis = 1)\
.sort_values(['postcode', 'binInfo_bin_type'])\
.rename(columns = {'i':'collection_time'})\
.reset_index(drop = True)

insertQuery = 'INSERT INTO wasteCollection(binInfo_bin_type, postcode, collection_time) VALUES\n'
values = ''

for i in wasteCollectionDate.index: 
    values = values + f'("{wasteCollectionDate.binInfo_bin_type[i].strip()}", "{wasteCollectionDate.postcode[i].strip()}", "{wasteCollectionDate.collection_time[i].strip()}"),\n'.replace('"', "'")

with open('wasteCollection.sql', 'w+') as output: 
    output.write((insertQuery + values).rstrip(','))

output.close()

In [154]:
wasteCollectionDate.to_csv('wasteCollectionDate.csv', index = False)